# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -8.397865258029                   -0.90    5.2   27.4ms
  2   -8.400254872973       -2.62       -1.74    1.0   74.4ms
  3   -8.400401401372       -3.83       -2.95    1.2   19.8ms
  4   -8.400427785914       -4.58       -2.91    3.2   25.2ms
  5   -8.400427916290       -6.88       -3.00    1.0   19.3ms
  6   -8.400428145801       -6.64       -4.91    1.0   19.5ms
  7   -8.400428151670       -8.23       -4.36    3.2   25.6ms
  8   -8.400428152201       -9.27       -5.41    1.5   20.9ms
  9   -8.400428152209      -11.09       -6.48    1.5   49.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime 
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397857870858                   -0.90           5.5    1.70s
  2   -8.400383480118       -2.60       -1.78   0.80    2.0    494ms
  3   -8.400423121929       -4.40       -2.98   0.80    1.0    216ms
  4   -8.400428104666       -5.30       -3.47   0.80    2.5   20.4ms
  5   -8.400428148646       -7.36       -4.82   0.80    1.2   17.3ms
  6   -8.400428152196       -8.45       -5.50   0.80    2.8   98.4ms
  7   -8.400428152208      -10.91       -6.34   0.80    1.8   18.5ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

┌ Warning: x_tol is deprecated. Use x_abstol or x_reltol instead. The provided value (-1) will be used as x_abstol.
└ @ Optim ~/.julia/packages/Optim/gmigl/src/types.jl:110
┌ Warning: f_tol is deprecated. Use f_abstol or f_reltol instead. The provided value (-1) will be used as f_reltol.
└ @ Optim ~/.julia/packages/Optim/gmigl/src/types.jl:120
n     Energy            log10(ΔE)   log10(Δρ)   Δtime 
---   ---------------   ---------   ---------   ------
  1   +0.881248760093                   -1.10    3.38s
  2   -1.615942946854        0.40       -0.69    149ms
  3   -4.493268024145        0.46       -0.43    107ms
  4   -6.026182682454        0.19       -0.54   44.6ms
  5   -7.454225331712        0.15       -0.79   43.8ms
  6   -7.852230369761       -0.40       -1.31   32.3ms
  7   -8.135791246128       -0.55       -1.58   32.7ms
  8   -8.233809457770       -1.01       -1.81   32.5ms
  9   -8.311151394424       -1.11       -1.90   60.5ms
 10   -8.346268138903       -1.45       -2.03   3

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -8.397871169817                   -0.90    5.2   27.1ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime 
---   ---------------   ---------   ---------   ------
  1   -8.400427988683                   -1.79    7.91s
  2   -8.400428152209       -6.79       -4.04    3.10s
  3   -8.400428152209      -14.45       -7.85    108ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.7972932235666985e-7
|ρ_newton - ρ_scfv| = 5.151452502238823e-7
|ρ_newton - ρ_dm|   = 9.248056381029547e-7
